In [1]:
############# MNIST 데이터 로딩 및 신경망 정의##########################

# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

# Training settings
batch_size = 64

# MNIST Dataset
# MNIST 데이터 셋을 가져와 train, test 구분
train_dataset = datasets.MNIST(root='./data/',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)
test_dataset = datasets.MNIST(root='./data/',
                                train=False,
                                transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
# 각 데이터 셋에서 용도에 맞게 데이터를 랜덤하게 섞어서 Batch size 수에 맞게 가져온다
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=batch_size,
                                            shuffle=False)

# 신경망 구조 정의
class Net(nn.Module):
    
    # 초기화
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1_out_np = np.zeros((1, 3, 24, 24))
        self.mp1_out_np = np.zeros((1, 3, 12, 12))
        self.conv2_out_np = np.zeros((1, 3, 8, 8))
        self.mp2_out_np = np.zeros((1, 3, 4, 4))
        self.fc_in_np = np.zeros((1, 48))
        self.fc_out_np = np.zeros((1, 10))
        
        # 커널 크기가 5, 입력 채널이 1, 출력 채널이 3인 CNN 층 구성
        # 출력 채널은 커널의 개수를 의미
        self.conv1 = nn.Conv2d(1, 3, kernel_size=5) # Num of weight = 
        
        # 커널 크기가 5, 입력 채널이 3, 출력 채널이 3인 CNN 층 구성
        self.conv2 = nn.Conv2d(3, 3, kernel_size=5)
        
        # Max Pooling Layer, 파라미터는 kernel size 의미, 4개의 값 중 최댓값을 출력한다.
        self.mp = nn.MaxPool2d(2)
        
        # Fully Connected Layer
        self.fc_1 = nn.Linear(48, 10) # Num of Weight = 480
        
    def forward(self, x):
        # 텐서를 일자로 펴기 위해 x의 한 원소의 크기 계산
        in_size = x.size(0)
        
        # CNN 층 출력이 최대 풀링 층을 지나 활성함수 ReLU를 지난다.
        x = self.conv1(x)
        self.conv1_out_np = x.detach().numpy()
        
        x = F.relu(self.mp(x))
        self.mp1_out_np = x.detach().numpy()

        x = self.conv2(x)
        self.conv2_out_np = x.detach().numpy()
        
        x = F.relu(self.mp(x))
        self.mp2_out_np = x.detach().numpy()
        
        # 텐서를 Fully Connected Layer에 넣기 위해 일자로 편다
        x = x.view(in_size, -1) # flatten the tensor
        self.fc_in_np = x.detach().numpy()
        
        # 일자로 편 텐서로 Fully Connected Layer 계산
        x = self.fc_1(x)
        self.fc_out_np = x.detach().numpy()
        
        # 출력층의 활성함수는 Softmax 사용
        return F.log_softmax(x)
    
# Instantiation    
model = Net()

# 최적화 방법으로 Momentum(= 0.5) 방식을 추가한 Stochastical Gradient Descent를 사용한다.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

################### 신경망 학습 및 실행 ##################################

def train(epoch):
    # 해당 모델을 training 상태로 설정하는 함수
    model.train()
    
    # train_loader로 batch_idx, data, target 각각 반복마다 
    # train_loader의 인덱스를 하나씩 늘려가며 data를 넣는다.
    for batch_idx, (data, target) in enumerate(train_loader):
        
        #print(batch_idx)
        
        # Pytorch에서 다룰 수 있게 Variable로 변경
        data, target = Variable(data), Variable(target)
        
        #print(np.shape(data))
        
        # 다음 최적화를 위해 전에 저장되어있던 값을 초기화
        optimizer.zero_grad()
        
        # model을 feed-forward 되어 나온 출력값
        output = model(data)
        
        # Negative Log Likelihood 로 Loss를 계산 
        loss = F.nll_loss(output, target)
        
        # Gradient 값을 계산한다.
        loss.backward()
        
        # back propagation을 통한 가중치값 최적화를 한번 진행
        optimizer.step()
        
        # batch_idx가 10의 배수일 때 => 10번마다
        if batch_idx % 10 == 0:
            #각 값을 모니터 출력
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                # Percentage로 나타내기 위해 100을 곱함
                100. * batch_idx / len(train_loader), loss.item()))
                # data[0] is deprecated

# 테스트에 사용할 함수 정의
def test():
    # 해당 모델을 evaluation 상태로 설정하는 함수
    model.eval()
    # loss 값 stacking 위해 초기화
    test_loss = 0
    # 맞춘 수를 stacking 위해 초기화
    correct = 0
    
    
    for data, target in test_loader:
        
        # 각 배열을 PyTorch로 다룰 수 있는 Variable로 만들어준다
        # volatile was removed and now has no effect
        data, target = Variable(data, volatile=True), Variable(target)
        
        # model을 feed-forward 되어 나온 출력값
        output = model(data)
        
        # sum up batch loss
        #loss 값 stacking
        test_loss += F.nll_loss(output, target, size_average=False).item()
        
        # get the index of the max log-probability
        # max() : 1과 출력 data 중 최대값 출력
        pred = output.data.max(1, keepdim=True)[1]
        
        # view_as(pred) == view(pred.size())
        # => 같은 데이터를 갖는 텐서 배열을 1*n 의 새로운 배열의 텐서로 반환한다.
        # cpu() : CUDA 처리가 불가능하여 CPU로 처리하게 하는 함수
        # sum() : 텐서의 모든 원소 합 반환
        # eq() : 입력받은 두 배열을 비교하여 대응되는 원소값이 갇으면 1,
        # 다르면 0을 갖는 같은 크기의 배열을 반환한다
        # pred와 target 비교하여 둘이 같으면 1을 갖는 배열의 합, 즉 pred 와 target이 맞는 수를 센다 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    # 데이터 셋의 크기로 Loss를 나눠준다    
    test_loss /= len(test_loader.dataset)
    
    # 각 결과값을 모니터 출력
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 10):
    train(epoch)
    test()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.332074
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.344449
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.294566
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.320626
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.300294
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.300256
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.275375
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.273973
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.278428
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.271183
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.247332
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.208678
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.223641
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.146708
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.122668
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.993432
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.875436
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.754396
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.376826
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.243472
Train Epoch: 1 [12800/60000 (

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:153: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
D:\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.3250, Accuracy: 9028/10000 (90%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.455013
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.238540
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.296613
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.308804
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.285329
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.401596
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.422094
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.343862
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.434531
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.325801
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.177775
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.377713
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.416484
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.388439
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.629142
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.221194
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.349990
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.375740
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.297942
Train Epoch: 2 [12

Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.160352
Train Epoch: 5 [27520/60000 (46%)]	Loss: 0.065645
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.150738
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.113210
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.158056
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.233650
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.190896
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.132681
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.289054
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.204568
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.155830
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.185546
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.189577
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.115025
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.110519
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.439417
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.412079
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.133531
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.179257
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.366123


Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.201556
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.082694
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.214892
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.221405
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.088034
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.283713
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.055034
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.095313
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.132579

Test set: Average loss: 0.1312, Accuracy: 9580/10000 (95%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.170572
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.117488
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.162322
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.133190
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.086222
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.216163
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.100196
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.193285
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.115319
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.101174
Train Epoch:

In [9]:
################# 샘플 통과 후 결과값 추출 #################
from PIL import Image

# 샘플을 이용한 테스트
def sample_test():
    # 해당 모델을 evaluation 상태로 설정하는 함수
    model.eval()
    # loss 값 stacking 위해 초기화
    test_loss = 0
    # 맞춘 수를 stacking 위해 초기화
    correct = 0
    
    ############## 테스트 샘플 (0 한장) 불러오기 ####################
    
    # target 생성 (label : 0)
    target = Variable(torch.tensor([0]))
    
    # data 생성 (0_0.bmp)
    img = Image.open("C:\\Users\\kilin\\Desktop\\MNIST_CNN\\TEST_MNIST_SAMPLE\\0_0.bmp", "r")
    np_img = np.array(img)
    np_img_re = np.reshape(np_img, (1,1,28,28))
    
    # 0 - 255 => 0 - 1 로 정규화, np.array => tensor 변환
    data = Variable(torch.tensor((np_img_re / 255), dtype = torch.float32))
    
    # model을 feed-forward 되어 나온 출력값
    output = model(data)
    #print(np.shape(output))
    
    # sum up batch loss
    # loss 값 stacking
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    
    # get the index of the max log-probability
    # max() : 1과 출력 data 중 최대값 출력
    pred = output.data.max(1, keepdim=True)[1]
    
    # view_as(pred) == view(pred.size())
    # => 같은 데이터를 갖는 텐서 배열을 1*n 의 새로운 배열의 텐서로 반환한다.
    # cpu() : CUDA 처리가 불가능하여 CPU로 처리하게 하는 함수
    # sum() : 텐서의 모든 원소 합 반환
    # eq() : 입력받은 두 배열을 비교하여 대응되는 원소값이 갇으면 1,
    # 다르면 0을 갖는 같은 크기의 배열을 반환한다
    # pred와 target 비교하여 둘이 같으면 1을 갖는 배열의 합, 즉 pred 와 target이 맞는 수를 센다 
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    print(correct)
    
    # 데이터 셋의 크기로 Loss를 나눠준다    
    #test_loss /= len(test_loader.dataset)
    
    # 각 결과값을 모니터 출력
    #print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        #test_loss, correct, len(test_loader.dataset),
        #100. * correct / len(test_loader.dataset)))
    
sample_test()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(1)


In [10]:
############## Conv1 가중치 값 HEX 추출 ############

# float => int
int_conv1_weight_1 =  torch.tensor((model.conv1.weight.data[0][0] * 128), dtype = torch.int32)
int_conv1_weight_2 =  torch.tensor((model.conv1.weight.data[1][0] * 128), dtype = torch.int32)
int_conv1_weight_3 =  torch.tensor((model.conv1.weight.data[2][0] * 128), dtype = torch.int32)
int_conv1_bias = torch.tensor((model.conv1.bias.data * 128), dtype = torch.int32)

print("Signed")
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)
print(int_conv1_bias)

# signed int => unsigned int
for i in range(5):
    for j in range(5):
        if int_conv1_weight_1[i][j] < 0:
            int_conv1_weight_1[i][j] += 256
        if int_conv1_weight_2[i][j] < 0:
            int_conv1_weight_2[i][j] += 256
        if int_conv1_weight_3[i][j] < 0:
            int_conv1_weight_3[i][j] += 256

for i in range(3):
    if int_conv1_bias[i] < 0:
            int_conv1_bias[i] += 256

print ("Unsigned")
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)
print(int_conv1_bias)

np.savetxt('conv1_weight_1.txt', int_conv1_weight_1, fmt='%1.2x',delimiter = " ")
np.savetxt('conv1_weight_2.txt', int_conv1_weight_2, fmt='%1.2x',delimiter = " ")
np.savetxt('conv1_weight_3.txt', int_conv1_weight_3, fmt='%1.2x',delimiter = " ")
np.savetxt('conv1_bias.txt', int_conv1_bias, fmt='%1.2x',delimiter = " ")

Signed
tensor([[-25,  16,  -1,  29,  19],
        [-11,  -3,   1,  43,  31],
        [ 13,   5,  62,  56,  23],
        [ -3,  34, 105,  67,   4],
        [ -1,  43,  52,  74,  45]], dtype=torch.int32)
tensor([[ -8, -27, -41,  -7, -31],
        [ -7, -30, -80, -63, -61],
        [ 56, -12, -25, -25,   0],
        [ 68,  61,  96,  62,  19],
        [ 28,  53,  56,  58,  68]], dtype=torch.int32)
tensor([[ 55,  36,  33, -13, -30],
        [ 40,   7,  14,   5, -18],
        [ 37,  25,  -9,   7, -19],
        [  0,  18,   1,  33,  61],
        [ 12,  40,  53,  34,  62]], dtype=torch.int32)
tensor([  1, -20,   0], dtype=torch.int32)
Unsigned
tensor([[231,  16, 255,  29,  19],
        [245, 253,   1,  43,  31],
        [ 13,   5,  62,  56,  23],
        [253,  34, 105,  67,   4],
        [255,  43,  52,  74,  45]], dtype=torch.int32)
tensor([[248, 229, 215, 249, 225],
        [249, 226, 176, 193, 195],
        [ 56, 244, 231, 231,   0],
        [ 68,  61,  96,  62,  19],
        [ 28,  53,  5

In [11]:
############## Conv2 가중치 값 HEX 추출 ############

print(np.shape(model.conv2.weight))

# float => int
int_conv2_weight_11 =  torch.tensor((model.conv2.weight.data[0][0]* 128), dtype = torch.int32)
int_conv2_weight_12 =  torch.tensor((model.conv2.weight.data[0][1]* 128), dtype = torch.int32)
int_conv2_weight_13 =  torch.tensor((model.conv2.weight.data[0][2]* 128), dtype = torch.int32)

int_conv2_weight_21 =  torch.tensor((model.conv2.weight.data[1][0] * 128), dtype = torch.int32)
int_conv2_weight_22 =  torch.tensor((model.conv2.weight.data[1][1] * 128), dtype = torch.int32)
int_conv2_weight_23 =  torch.tensor((model.conv2.weight.data[1][2] * 128), dtype = torch.int32)

int_conv2_weight_31 =  torch.tensor((model.conv2.weight.data[2][0] * 128), dtype = torch.int32)
int_conv2_weight_32 =  torch.tensor((model.conv2.weight.data[2][1] * 128), dtype = torch.int32)
int_conv2_weight_33 =  torch.tensor((model.conv2.weight.data[2][2] * 128), dtype = torch.int32)

int_conv2_bias = torch.tensor((model.conv2.bias.data * 128), dtype = torch.int32)

print ("Signed")
print(int_conv2_weight_11)
print(int_conv2_weight_12)
print(int_conv2_weight_13, '\n')

print(int_conv2_weight_21)
print(int_conv2_weight_22)
print(int_conv2_weight_23, '\n')

print(int_conv2_weight_31)
print(int_conv2_weight_32)
print(int_conv2_weight_33, '\n')

print(int_conv2_bias)


# signed int => unsigned int
for i in range(5):
    for j in range(5):
        if int_conv2_weight_11[i][j] < 0:
            int_conv2_weight_11[i][j] += 256
        if int_conv2_weight_12[i][j] < 0:
            int_conv2_weight_12[i][j] += 256
        if int_conv2_weight_13[i][j] < 0:
            int_conv2_weight_13[i][j] += 256
            
        if int_conv2_weight_21[i][j] < 0:
            int_conv2_weight_21[i][j] += 256
        if int_conv2_weight_22[i][j] < 0:
            int_conv2_weight_22[i][j] += 256
        if int_conv2_weight_23[i][j] < 0:
            int_conv2_weight_23[i][j] += 256
            
        if int_conv2_weight_31[i][j] < 0:
            int_conv2_weight_31[i][j] += 256
        if int_conv2_weight_32[i][j] < 0:
            int_conv2_weight_32[i][j] += 256
        if int_conv2_weight_33[i][j] < 0:
            int_conv2_weight_33[i][j] += 256

for i in range(3):
    if int_conv2_bias[i] < 0:
            int_conv2_bias[i] += 256

print ("Unsigned")
print(int_conv2_weight_11)
print(int_conv2_weight_12)
print(int_conv2_weight_13, '\n')

print(int_conv2_weight_21)
print(int_conv2_weight_22)
print(int_conv2_weight_23, '\n')

print(int_conv2_weight_31)
print(int_conv2_weight_32)
print(int_conv2_weight_33, '\n')

print(int_conv2_bias)

np.savetxt('conv2_weight_11.txt', int_conv2_weight_11, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_12.txt', int_conv2_weight_12, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_13.txt', int_conv2_weight_13, fmt='%1.2x',delimiter = " ")

np.savetxt('conv2_weight_21.txt', int_conv2_weight_21, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_22.txt', int_conv2_weight_22, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_23.txt', int_conv2_weight_23, fmt='%1.2x',delimiter = " ")

np.savetxt('conv2_weight_31.txt', int_conv2_weight_31, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_32.txt', int_conv2_weight_32, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_33.txt', int_conv2_weight_33, fmt='%1.2x',delimiter = " ")

np.savetxt('conv2_bias.txt', int_conv2_bias, fmt='%1.2x',delimiter = " ")


torch.Size([3, 3, 5, 5])
Signed
tensor([[ 20,  60,   2,  -5, -24],
        [ 28,  48,  -5,   2,   6],
        [ 20,  26,  -1,  25,   9],
        [ -2,   1, -29, -24, -20],
        [-37, -47, -57, -24, -36]], dtype=torch.int32)
tensor([[-35, -31,   4,  -8,  -7],
        [-47, -23,  11,  19,  34],
        [-63,  -6,  72,  43,  36],
        [-30,  30,  49,  14,   2],
        [-37,  20,  30,   4,  -3]], dtype=torch.int32)
tensor([[ 21,  21,  10,  -8, -10],
        [  5,  19,  47,  34,  38],
        [ 17,  14,  63,  40,  25],
        [  5,  25,  15,   9, -10],
        [-17, -21,   0,   4, -27]], dtype=torch.int32) 

tensor([[  4,  11,   4, -10,   4],
        [-15, -20,  -5,   0,   6],
        [  2, -31, -51, -29, -16],
        [ 12,  24,   2, -15, -41],
        [-20,  16,  10,  18,   8]], dtype=torch.int32)
tensor([[-27, -13,  -6,  -9, -18],
        [ -3,   0,   0,   5,  15],
        [ 16,   6, -17,  -5,  -9],
        [ 23,  56,  65,  27,  13],
        [-14,  18,  61,  60,  87]], dtype=torc

In [12]:
############## FC Layer가중치 값 HEX 추출 ############

print(np.shape(model.fc_1.weight))
print((model.fc_1.weight * 128).int())

print(np.shape(model.fc_1.bias))
print((model.fc_1.bias * 128).int())

int_fc_weight = (model.fc_1.weight * 128).int()
int_fc_bias = (model.fc_1.bias * 128).int()

# signed int => unsigned int
for i in range(10):
    for j in range(48):
        if int_fc_weight[i][j] < 0 :
            int_fc_weight[i][j] += 256
    if int_fc_bias[i] < 0 :
        int_fc_bias[i] += 256
        
print(int_fc_weight)
print(int_fc_bias)

np.savetxt('fc_weight.txt', int_fc_weight, fmt='%1.2x',delimiter = " ")
np.savetxt('fc_bias.txt', int_fc_bias, fmt='%1.2x',delimiter = " ")

torch.Size([10, 48])
tensor([[-14,  13,  22,  10, -16, -14, -26,  19,   1,  33, -18,   4,  41,  40,
          -1, -14,  17,  -2,  -9,   4,  -3, -10, -42, -41, -15,  -3, -19, -12,
          14,  26,  27, -24, -12,  11,   3,  -1,  14, -36, -49,  -8,  -7, -40,
           2,  46,   6,  -4,  19,  27],
        [  3,  -7, -24,  -4,   5, -40,  58, -11, -18, -31,  49,  10,   0, -33,
          52, -28, -13,  -5,   1, -57, -29, -22, -18, -29,  -2,   1, -26,  53,
           5, -41, -19,  44, -14, -15, -25,  23,  -5,  40, -18, -28,  -7,  33,
         -19,  18,  40, -14,   3,   8],
        [ 30,  35,   2, -27, -44, -37, -36,  19, -39, -44, -18,  18,  19,  47,
           1,   6,  45,  31,  -7, -24,   8,   5,   4,   4,  14, -33,   1,  60,
          -9,   9,   8,  50,  15, -10, -24, -56,  23, -10,   5,  -3,  28,   8,
          32,  48,   3, -17, -24,  24],
        [ 67,  16,  -8,   4, -22, -45,  13,   8,   3,  -5,  13, -27, -27, -34,
         -31,  -3, -34,  -3,   0, -15,   5,  22,  34,  30,  55,  10, 

In [13]:
######################### 신경망 통과 값 추출 ###########################
print(np.shape(model.conv1_out_np))
np.savetxt('out_conv1_value_1.txt', model.conv1_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv1_value_2.txt', model.conv1_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv1_value_3.txt', model.conv1_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.mp1_out_np))
np.savetxt('out_mp1_value_1.txt', model.mp1_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp1_value_2.txt', model.mp1_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp1_value_3.txt', model.mp1_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.conv2_out_np))
np.savetxt('out_conv2_value_1.txt', model.conv2_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv2_value_2.txt', model.conv2_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_conv2_value_3.txt', model.conv2_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.mp2_out_np))
np.savetxt('out_mp2_value_1.txt', model.mp2_out_np[0][0]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp2_value_2.txt', model.mp2_out_np[0][1]*128, fmt='%1.5d',delimiter = " ")
np.savetxt('out_mp2_value_3.txt', model.mp2_out_np[0][2]*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.fc_in_np))
np.savetxt('fc_in_value.txt', model.fc_in_np*128, fmt='%1.5d',delimiter = " ")

print(np.shape(model.fc_out_np))
np.savetxt('fc_out_value.txt', model.fc_out_np*128, fmt='%1.5d',delimiter = " ")

(1, 3, 24, 24)
(1, 3, 12, 12)
(1, 3, 8, 8)
(1, 3, 4, 4)
(1, 48)
(1, 10)


In [14]:
# 원래 입력 / 가중치 / 바이어스 출력
np.set_printoptions(suppress=True)
np.set_printoptions(precision=6)

print("Original Input")
print(model.mp1_out_np[0])
orig_input_1 = model.mp1_out_np[0][0]
orig_input_2 = model.mp1_out_np[0][1]
orig_input_3 = model.mp1_out_np[0][2]

print("Original Weight")
print(model.conv2.weight.detach().numpy())
orig_weight = model.conv2.weight.detach().numpy()

print("Original Bias")
print(model.conv2.bias.detach().numpy())
orig_bias = model.conv2.bias.detach().numpy()

orig_output_calc_1 = np.zeros((3,8,8))
orig_output_calc_2 = np.zeros((3,8,8))
orig_output_calc_3 = np.zeros((3,8,8))
orig_output_calc = np.zeros((3,8,8))

for c in range(3):
    for i in range(8):
        for j in range(8):
            orig_output_calc_1[c][i][j] += (orig_input_1[i:i+5, j:j+5] * orig_weight[c][0]).sum()
            orig_output_calc_2[c][i][j] += (orig_input_2[i:i+5, j:j+5] * orig_weight[c][1]).sum()
            orig_output_calc_3[c][i][j] += (orig_input_3[i:i+5, j:j+5] * orig_weight[c][2]).sum()         
            orig_output_calc[c][i][j] = orig_output_calc_1[c][i][j] + orig_output_calc_2[c][i][j] + orig_output_calc_3[c][i][j] + orig_bias[c]
        
print("\nBias : ")
print(orig_bias)
print("\nCalc Value :\n")
print(orig_output_calc) 
print("\nReal Value :\n")
print(model.conv2_out_np[0])

Original Input
[[[0.007946 0.007946 0.007946 0.007946 0.485303 2.433931 2.86805
   0.795214 0.007946 0.007946 0.007946 0.007946]
  [0.007946 0.007946 0.007946 0.526232 2.294008 4.878462 4.494352
   1.776778 0.269568 0.007946 0.007946 0.007946]
  [0.007946 0.007946 0.262226 2.138685 4.997587 5.42438  5.123318
   3.718413 2.462729 0.392078 0.007946 0.007946]
  [0.007946 0.052655 1.224695 4.752855 5.336329 5.290852 4.306835
   3.687219 3.75194  1.586619 0.145947 0.007946]
  [0.007946 0.087386 2.709602 5.223812 4.915937 3.461112 1.492861
   2.755593 3.816344 3.520272 0.83291  0.007946]
  [0.007946 0.289716 3.459296 4.596095 3.143979 0.517921 0.
   1.717676 4.691989 4.296669 1.12771  0.010092]
  [0.007946 0.49768  4.383057 4.611019 0.754726 0.       0.737615
   3.018578 5.127934 4.624765 0.979043 0.03125 ]
  [0.007946 0.789257 4.539495 4.514042 1.136255 1.673958 3.273121
   5.175742 5.172872 3.462619 0.049458 0.007946]
  [0.007946 0.688944 4.365803 4.677119 3.816629 4.618104 5.225662
   5.2

   -4.26109 ]]]

Real Value :

[[[-0.828657  0.704893  1.951454  4.398038  7.253537  7.254332  2.941462
   -1.957195]
  [ 0.741294  1.855782  4.447139  9.646738 12.662814 11.908775  6.842013
    1.550577]
  [ 0.119945  1.614863  4.51439   7.893129 10.060531 10.734888  8.299784
    4.94633 ]
  [-1.64158   0.571877  4.172332  4.380099  3.920083  4.610086  6.142311
    6.784405]
  [-2.08283   1.332775  4.652186  1.525881 -0.761069  0.436467  3.622389
    7.281419]
  [ 0.116472  5.234763  9.012963  4.75481   0.47695   0.313369  3.093289
    7.89415 ]
  [ 4.854158 12.622501 16.285841  9.297841  4.304612  3.844903  5.803882
    8.757852]
  [ 6.964129 14.440686 16.891525 11.215445  7.455584  7.052281  8.527936
    8.914791]]

 [[ 1.146417  0.586187 -1.169071 -2.942539 -1.610942  2.453888  7.262853
    8.690933]
  [-1.834294 -2.29562  -3.389921 -5.245375 -6.100524 -3.842926  0.442187
    4.9631  ]
  [-1.033363 -2.016812 -2.564297 -2.824316 -2.422157 -2.913472 -2.215978
   -0.722873]
  [ 1.7221

In [15]:
np.set_printoptions(precision=0)

print("Convolution Input")
print(np.shape(model.mp1_out_np))
_input = model.mp1_out_np[0] * 128
print(_input)
print('\n')

print("Convolution Weight")
print(np.shape(model.conv2.weight.detach().numpy()))
weight = model.conv2.weight.detach().numpy() * 128
print(weight)

print("Convolution Bias")
print(np.shape(model.conv2.bias.detach().numpy()))
bias = model.conv2.bias.detach().numpy() * 128
print(bias)

print("Convolution Output")
print(np.shape(model.conv2_out_np))
output = model.conv2_out_np * 128
print(output)

Convolution Input
(1, 3, 12, 12)
[[[  1.   1.   1.   1.  62. 312. 367. 102.   1.   1.   1.   1.]
  [  1.   1.   1.  67. 294. 624. 575. 227.  35.   1.   1.   1.]
  [  1.   1.  34. 274. 640. 694. 656. 476. 315.  50.   1.   1.]
  [  1.   7. 157. 608. 683. 677. 551. 472. 480. 203.  19.   1.]
  [  1.  11. 347. 669. 629. 443. 191. 353. 488. 451. 107.   1.]
  [  1.  37. 443. 588. 402.  66.   0. 220. 601. 550. 144.   1.]
  [  1.  64. 561. 590.  97.   0.  94. 386. 656. 592. 125.   4.]
  [  1. 101. 581. 578. 145. 214. 419. 662. 662. 443.   6.   1.]
  [  1.  88. 559. 599. 489. 591. 669. 677. 603. 136.   0.   1.]
  [  1.  55. 352. 520. 611. 662. 654. 522. 219.   0.   1.   1.]
  [  1.  14. 146. 262. 405. 433. 314. 129.   2.   0.   1.   1.]
  [  1.   1.  19.  87. 114.  73.  31.   0.   1.   1.   1.   1.]]

 [[  0.   0.   0.   0.  69. 304. 365. 213.   0.   0.   0.   0.]
  [  0.   0.   0.  57. 202. 348. 388. 300.  86.   0.   0.   0.]
  [  0.   0.  28. 211. 289. 280. 314. 506. 415. 146.   0.   0.]
  [  

In [16]:
print(np.shape(_input))
print(np.shape(weight))
print(np.shape(output))

_input_1 = _input[0]
_input_2 = _input[1]
_input_3 = _input[2]

output_calc_1 = np.zeros((3,8,8))
output_calc_2 = np.zeros((3,8,8))
output_calc_3 = np.zeros((3,8,8))
output_calc = np.zeros((3,8,8))

for c in range(3):
    for i in range(8):
        for j in range(8):
            output_calc_1[c][i][j] += (_input_1[i:i+5, j:j+5] * weight[c][0]).sum()
            output_calc_2[c][i][j] += (_input_2[i:i+5, j:j+5] * weight[c][1]).sum()
            output_calc_3[c][i][j] += (_input_3[i:i+5, j:j+5] * weight[c][2]).sum()
            output_calc[c][i][j] = output_calc_1[c][i][j] + output_calc_2[c][i][j] + output_calc_3[c][i][j] + bias[c] * 128
        
print("\nBias : ")
print(bias)
print("\nCalc Value :\n")
print(output_calc / 128) 
print("\nReal Value :\n")
print(output)

(3, 12, 12)
(3, 3, 5, 5)
(1, 3, 8, 8)

Bias : 
[-4. -6.  3.]

Calc Value :

[[[ -106.    90.   250.   563.   928.   929.   377.  -251.]
  [   95.   238.   569.  1235.  1621.  1524.   876.   198.]
  [   15.   207.   578.  1010.  1288.  1374.  1062.   633.]
  [ -210.    73.   534.   561.   502.   590.   786.   868.]
  [ -267.   171.   595.   195.   -97.    56.   464.   932.]
  [   15.   670.  1154.   609.    61.    40.   396.  1010.]
  [  621.  1616.  2085.  1190.   551.   492.   743.  1121.]
  [  891.  1848.  2162.  1436.   954.   903.  1092.  1141.]]

 [[  147.    75.  -150.  -377.  -206.   314.   930.  1112.]
  [ -235.  -294.  -434.  -671.  -781.  -492.    57.   635.]
  [ -132.  -258.  -328.  -362.  -310.  -373.  -284.   -93.]
  [  220.   374.   522.   452.   378.   128.    -8.  -228.]
  [  398.   921.  1310.  1186.   798.   411.   135.  -143.]
  [  -34.   426.   980.   984.   575.   167.   -74.  -113.]
  [ -749.  -695.  -445.  -217.  -185.  -292.  -411.  -273.]
  [ -589.  -831. -1024

In [17]:
np.set_printoptions(precision=0)
print("\nCalc Value 1 :\n")
print(output_calc_1) 

print("\nCalc Value 2 :\n")
print(output_calc_2) 

print("\nCalc Value 3 :\n")
print(output_calc_3) 

print("Sum")
print(output_calc_1 + output_calc_2 + output_calc_3)


Calc Value 1 :

[[[ -79563. -115480. -119358.  -83185.  -17614.    8089.  -19178.
    -43569.]
  [ -76744.  -95861.  -57902.   21259.   71909.   58105.    3008.
    -30303.]
  [ -77871.  -72264.    -109.   69870.   77375.   31645.   -8140.
    -15770.]
  [ -86202.  -63898.    8516.   29483.      24.  -51432.  -51033.
     -8752.]
  [ -99805.  -74930.  -25465.  -50144. -102209. -119513.  -67308.
     22131.]
  [ -88203.  -55248.  -29550.  -82149. -126760. -100607.  -18583.
     77434.]
  [ -40519.    7861.   28970.  -29656.  -44251.    1708.   71418.
    120942.]
  [    207.   57557.   83441.   56935.   63897.   92173.  121985.
    118717.]]

 [[ -34046.  -46650.  -68184.  -87613.  -83399.  -52588.   -9879.
     10516.]
  [ -49032.  -62741.  -77445.  -91922.  -91984.  -72616.  -38571.
     -7807.]
  [ -50142.  -56353.  -62881.  -67248.  -65087.  -66195.  -46464.
    -24871.]
  [ -39451.  -32929.  -31157.  -33193.  -32252.  -46073.  -45074.
    -38079.]
  [ -42196.  -27861.  -15240.  -1